In [1]:
from am4pa.linnea import LinneaConfig, DataManagerLinnea,OperandsSamplerBase
from am4pa.linnea import MeasurementsMaganer
from va4algs import RankingDataLinnea, RankingModel
import random
import os
import json
import pandas as pd
pd.set_option('display.max_rows', None)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
class OperandsSamplerNeighbour(OperandsSamplerBase):
    def __init__(self,num_operands,delta=50,seed=108):
        super().__init__(num_operands)
        self.seed = seed
        random.seed(self.seed)
        self.delta = delta
        self.focus = None
        
    def set_focus(self,focus):
        self.focus = focus
    
    def sample(self):
        
        if not self.focus:
            return
        
        ops = []
        
        for i in range(self.n):
            
            min_ = max(10,self.focus[i]-self.delta)
            max_ = self.focus[i]+self.delta
            
            ops.append(random.randint(min_,max_))
        return ops

In [38]:
class SmartMeasurementManager:
    def __init__(self,rdl, adj_risk_thresh=0.05):
        self.rdl = rdl
        self.dml = rdl.dml
        self.ranking_method = self.rdl.rm.method
        
        ret = self.rdl.load()
        assert ret != -1, "First rank data"
        print("Ranking data from {} has been loaded.".format(self.rdl.obj_path))
        
        self.history = {'focus':[],'dirty':False}
        self.history_file = os.path.join(self.dml.lc.local_dir,
                                         'history_{}.json'.format(self.rdl.obj_path.split('/')[-1].split('.pkl')[0]))
        if os.path.exists(self.history_file):
            with open(self.history_file, 'r') as jf:
                self.history = json.load(jf)
            
        
        self.im = None
        self.focus = []
        
        
        self.num_ops = len(self.rdl.data_anomalies['op_str'].tolist()[0].split('_'))
        self.osn = OperandsSamplerNeighbour(self.num_ops,delta=50)
        self.mm = MeasurementsMaganer(self.dml,self.osn,self.rdl.thread_str)
        
        self.adj_risk_thresh = adj_risk_thresh
        _ = self.filter_interesting_operands()
        
    
    def filter_interesting_operands(self):
        df = self.rdl.data_anomalies
        self.im = df[df['adj_risk']>self.adj_risk_thresh].sort_values(['rel-flops-cutoff','adj_risk'], ascending=[False,False])
            
            
        focus_ops = self.im[self.im['adj_risk']>self.adj_risk_thresh]['op_str'].tolist()
        
        self.focus = []
        for op_str in focus_ops:
            if op_str not in self.history['focus']:
                self.focus.append(op_str)
        
        return self.im
        
    def _smart_generate_variants(self):
        for op_str in self.focus:
            print("Focus: {}".format(op_str))
            self.osn.set_focus(list(map(int,op_str.split('_'))))
            self.mm.generate_variants_sampler(5)
           
    def generate_measure(self,reps,run_id,bSlrum):
        if self.history['dirty']:
            print("Call rank data before further measurements")
            return -1
        
        if not self.focus:
            print("Nothing to focus")
            return -1
            
        self._smart_generate_variants()
        self.mm.measure_variants(reps,run_id=run_id,bSlrum=bSlrum)
        self.history['dirty'] = True
        self.dml._update_json(self.history,self.history_file)
            
        return 1
        
    def check_slrum_status(self):
        self.dml.check_completed_slrum_jobs()
        if not self.dml.slrum_running_jobs['r']:
            print("Completed")
            return 1
        else:
            print(self.dml.slrum_running_jobs['r'])
            return -1
        
    def rank_update(self):
        if self.history['dirty']:
            self.rdl.rank3way(update=True)
            for op in self.focus:
                if not op in self.history['focus']:
                    self.history['focus'].append(op)
            self.history['dirty'] = False
            self.dml._update_json(self.history,self.history_file)
            self.rdl.save()
        else:
            print("No new measurements")
            
    def stop_search_path(self):
        if not self.history['dirty']:
            for op in self.focus:
                if not op in self.history['focus']:
                    self.history['focus'].append(op)
            self.dml._update_json(self.history,self.history_file)
            

In [39]:
EXPRESSION_FILE = 'gls.py'
EXPERIMENT_ID = 'GLS-ix01'

SERVER = "login18-1.hpc.itc.rwth-aachen.de"
UNAME = "as641651"
BACK_END_FOLDER = "~/PhD/va4algs-exps/"
FRONT_END_FOLDER = "codes/"

RANKING_METHOD = 3
THREADS = '24T'

bSLRUM = True
REPS = 10
RUN_ID = 1
ADJ_THRESH = 0.05


In [40]:
linnea_config = LinneaConfig(EXPRESSION_FILE)
linnea_config.set_local_directory("{}{}".format(FRONT_END_FOLDER,EXPERIMENT_ID))
linnea_config.setup_backend_details(server=SERVER,
                                   uname = UNAME,
                                   init_script="~/.analyzer",
                                   backend_root=BACK_END_FOLDER)
linnea_config.connect_backend()

In [41]:
dml = DataManagerLinnea(linnea_config)

{'problem': 'linnea-expressions/gls.py', 'backend': True, 'server': 'login18-1.hpc.itc.rwth-aachen.de', 'uname': 'as641651', 'local_dir': 'codes/GLS-ix01', 'backend_dir': '~/PhD/va4algs-exps/GLS-ix01'}


In [42]:
rm = RankingModel(name='R{}'.format(RANKING_METHOD),method=RANKING_METHOD)
rdl = RankingDataLinnea(dml,rm,THREADS)

In [43]:
sm = SmartMeasurementManager(rdl)

Ranking data from codes/GLS-ix01/ranking-data/rdl_24T_all_R3.pkl has been loaded.


In [44]:
sm.focus

[]

In [45]:
sm.history_file

'codes/GLS-ix01/history_rdl_24T_all_R3.json'

In [46]:
sm.filter_interesting_operands()

,op_str,num_algs,max-rel-flops,is_anomaly,n_rel0,n_anomalies,risk,time_inc,rel_inc,adj_risk,rel-flops-cutoff
60,304_247,100,0.737968,True,9,9,1.000000,0.000401,0.480986,0.480986,0.736185
62,337_249,100,0.733333,True,9,9,1.000000,0.000273,0.339285,0.339285,0.733333
51,338_206,100,0.700000,True,9,9,1.000000,0.000166,0.204306,0.204306,0.700000
45,378_172,100,0.637993,True,9,9,1.000000,0.000104,0.177785,0.177785,0.637993
74,717_82,100,0.122807,True,9,9,1.000000,0.000099,0.067785,0.067785,0.023392
57,740_50,100,0.048485,True,27,18,0.666667,0.000372,0.351967,0.234645,0.000000
31,699_47,100,0.043165,True,27,18,0.666667,0.000326,0.295212,0.196808,0.000000
56,672_35,100,0.033898,True,27,18,0.666667,0.000177,0.202571,0.135047,0.000000
73,649_17,100,0.021234,True,27,25,0.925926,0.000077,0.056578,0.052387,0.000000


In [47]:
sm.generate_measure(REPS,RUN_ID,bSLRUM)

Nothing to focus


-1

In [15]:
sm.check_slrum_status()

ls ~/PhD/va4algs-exps/GLS-ix01/experiments/24T/304_247/run_times_competing_1*
ls ~/PhD/va4algs-exps/GLS-ix01/experiments/24T/298_240/run_times_competing_1*
ls ~/PhD/va4algs-exps/GLS-ix01/experiments/24T/337_249/run_times_competing_1*
ls ~/PhD/va4algs-exps/GLS-ix01/experiments/24T/323_204/run_times_competing_1*
ls ~/PhD/va4algs-exps/GLS-ix01/experiments/24T/313_166/run_times_competing_1*
ls ~/PhD/va4algs-exps/GLS-ix01/experiments/24T/728_37/run_times_competing_1*
ls ~/PhD/va4algs-exps/GLS-ix01/experiments/24T/713_35/run_times_competing_1*
ls ~/PhD/va4algs-exps/GLS-ix01/experiments/24T/781_50/run_times_competing_1*
ls ~/PhD/va4algs-exps/GLS-ix01/experiments/24T/714_24/run_times_competing_1*
ls ~/PhD/va4algs-exps/GLS-ix01/experiments/24T/714_20/run_times_competing_1*
ls ~/PhD/va4algs-exps/GLS-ix01/experiments/24T/693_64/run_times_competing_1*
ls ~/PhD/va4algs-exps/GLS-ix01/experiments/24T/635_78/run_times_competing_1*
ls ~/PhD/va4algs-exps/GLS-ix01/experiments/24T/663_59/run_times_competi

1

In [48]:
sm.history

{'focus': ['378_172',
  '699_47',
  '304_247',
  '337_249',
  '338_206',
  '717_82',
  '740_50',
  '672_35',
  '649_17'],
 'dirty': False}

In [49]:
sm.rank_update()

No new measurements


In [50]:
df = rdl.data_anomalies
df

,op_str,num_algs,max-rel-flops,is_anomaly,n_rel0,n_anomalies,risk,time_inc,rel_inc,adj_risk,rel-flops-cutoff
0,1000_10,100,0.002907,False,27,0,0.000000,0.000000,0.000000,0.000000,0.000000
1,1000_1000,3,31.884903,False,1,0,0.000000,0.000000,0.000000,0.000000,0.000000
2,100_100,3,31.023290,False,1,0,0.000000,0.000000,0.000000,0.000000,0.000000
3,10_10,3,24.541619,False,1,0,0.000000,0.000000,0.000000,0.000000,0.000000
4,10_1000,100,0.026087,False,13,0,0.000000,0.000000,0.000000,0.000000,0.000000
5,1204_278,100,0.366972,False,9,0,0.000000,0.000000,0.000000,0.000000,0.000000
6,13_1381,100,0.025358,False,88,0,0.000000,0.000000,0.000000,0.000000,0.000000
7,13_440,100,0.073248,False,13,0,0.000000,0.000000,0.000000,0.000000,0.000000
8,1500_1500,3,32.022222,False,1,0,0.000000,0.000000,0.000000,0.000000,0.000000
9,223_47,100,0.343373,False,9,0,0.000000,0.000000,0.000000,0.000000,0.000000


In [51]:
print("Percent Anomalies: {}".format(rdl.percent_anomalies))
print("Average Risk: {}".format(rdl.avg_risk))
print("Average Adjusted Risk: {}".format(rdl.avg_adj_risk))

Percent Anomalies: 0.17333333333333334
Average Risk: 0.1150624881291548
Average Adjusted Risk: 0.02630092417615157


In [54]:
sm.stop_search_path()

In [55]:
sm.history

{'focus': ['378_172',
  '699_47',
  '304_247',
  '337_249',
  '338_206',
  '717_82',
  '740_50',
  '672_35',
  '649_17'],
 'dirty': False}